In [1]:
import numpy as np 
import random 
import re 
import tensorflow as tf 
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing 
import scipy.io as sio
import os

# encoding: UTF-8

from time import time
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
import matplotlib.pyplot as plt


from PIL import Image

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
#读取图片获得训练集和测试集
def get_files(file_dir,lable):
    X = []
    Y = []
    a=0

    img=np.zeros(shape=(32,32), dtype=np.int16)
    for file in os.listdir(file_dir):
        
        img=np.array(Image.open(file_dir+file))

        X.insert(a,img)
        Y.append(lable)
        a +=1
        
    return X,Y
        #print(x)

In [3]:

correct_lable = [0,1]
wrong_lable =[1,0]

X_train_correct, Y_train_correct = get_files('data\\train_correct_resize\\',correct_lable)

X_train_wrong, Y_train_wrong = get_files('data\\train_wrong_resize\\',wrong_lable)

X_test_correct, Y_test_correct = get_files('data\\test_correct_resize\\',correct_lable)

X_test_wrong, Y_test_wrong = get_files('data\\test_wrong_resize\\',wrong_lable)

X1 = []
Y1 = [] 

X1.extend(X_train_correct)
X1.extend(X_train_wrong)
X1.extend(X_test_correct)
X1.extend(X_test_wrong)


Y1.extend(Y_train_correct)
Y1.extend(Y_train_wrong)
Y1.extend(Y_test_correct)
Y1.extend(Y_test_wrong)


# 训练集和测试集的随机分割
x_train, x_test, y_train, y_test = train_test_split(X1 ,Y1, test_size=0.15,random_state=42) 
# x_train, x_test, y_train, y_test = train_test_split(X_train_correct + X_train_wrong + X_test_correct + X_test_wrong,
#                                                     Y_train_correct + Y_train_wrong + Y_test_correct + Y_test_wrong,
#                                                     test_size=0.15,random_state=42) 



In [36]:
#初始化权值
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1,seed=1) #生成一个截断的正态分布
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#初始化偏置
def bias_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1,seed=1)
    return tf.Variable(initial)
    #return tf.Variable(tf.zeros(shape))

#卷积层
def conv2d(x,W):
    #input tensor of shape [batch, in_height, in_width, in_channels]
    #filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #strides[0]=strides[3]=1. strides[1]代表ｘ方向的步长，strids[2]代表ｙ方向的步长
    #padding: A string from "SAME", "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#池化层
def max_pool(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')



def multilayer_perceptron(x_train, x_test, y_train, y_test): 

    x_test = np.reshape(x_test, (-1, 1024))
    x_train_size = np.size(x_train, 0)
    #定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 1024]) 
    y = tf.placeholder(tf.float32, [None, 2])
    
    #改变x的格式转为４Ｄ的向量【batch, in_height, in_width, in_channels]
    x_image = tf.reshape(x,[-1, 32, 32, 1])

    #初始化第一个卷积层的权值和偏量
    W_conv1 = weight_variable([5,5,1,64])#5*5的采样窗口，３２个卷积核从4个平面抽取特征
    b_conv1 = bias_variable([64])#每一个卷积核一个偏置值

    #把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool(h_conv1)#进行max-pooling,12-by-40
    
#     #初始化第二个卷积层的权值和偏置
#     W_conv2 = weight_variable([3,3,16,32]) #5*5的采样窗口，64个卷积核从32个平面抽取特征
#     b_conv2 = bias_variable([32]) #每一个卷积核一个偏置值

#     #把H_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
#     h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
#     h_pool2 = max_pool(h_conv2)#6-by-5
    

    '''  
    23*20的图片第一次卷积后还是100*240第一次池化后变为50*120
    第二次卷积后为50*120,第二次池化后变为25*60
    进过上面操作后得到32张25*60的平面
    '''

    #初始化第一全链接层的权值
    W_fc1 = weight_variable([16*16*64,256]) #上一层有6*10*64个神经元,全连接层有1024个神经元
    b_fc1 = bias_variable([256])

    #把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool1,[-1,16*16*64])
    #求第一个全连接层的输出
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

    #keep_prob用了表示神经元的输出概率
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    #初始化第二个全连接层
    W_fc2 = weight_variable([256,2])
    b_fc2 = bias_variable([2])

    
    #计算输出
    #sigmoid
    prediction = tf.nn.sigmoid(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)
    # 结果存放在一个布尔列表中
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    
    #交叉熵代价函数
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
    
    #使用AdamOptimizer进行优化
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    #求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))




    with tf.Session() as sess: 
        sess.run(tf.global_variables_initializer())
        #初始化参数 
        # Training cycle 
        for epoch in range(10000): 
            batch_index = random.randint(0, 1000) 
            #每一次从0-1000随机选出一个数，作为起始点 
            batch_data = x_train[batch_index:batch_index + 20] 
            batch_labels = y_train[batch_index:batch_index + 20] 
            #在上面所选的起始点开始，拿出32组数据进行一次训练 
            batch_data = np.reshape(batch_data, (-1, 1024))
            sess.run(optimizer,feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.7})
            Accuracy,Loss = sess.run((accuracy,cost),feed_dict={x: batch_data, y: batch_labels, keep_prob: 0.7})
            
            print("Accuracy:", Accuracy)
            print("Loss:", Loss)
            print('')
            #进行训练 
#             if epoch%300 == 0: 
#                 print(epoch)
# #                 accuracy = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
# #                 print("Accuracy:", sess.run(accuracy, {x:x_test,y: y_test, keep_prob: 1.0}))
# #                 print("Loss:", sess.run(cost, {x:x_test,y: y_test, keep_prob: 1.0}))
#                 Accuracy,Loss = sess.run((accuracy,cost) ,{x:x_test,y: y_test, keep_prob: 1.0})
                
#                 print("Accuracy:", Accuracy)
#                 print("Loss:", Loss)

                #每隔300次 输出对应的次数以及测试集所测试的准确度 
        # Test model 
        
        print("Total Accuracy:", accuracy.eval({x:x_test, y:y_test, keep_prob: 1.0})) 
        #最后，当训练结束时，输出最终的准确率。
 

In [38]:
#学习率设置
learning_rate = 1e-4

multilayer_perceptron(x_train, x_test, y_train, y_test)

Accuracy: 0.45
Loss: 0.6601872

Accuracy: 0.6
Loss: 0.69622153

Accuracy: 0.35
Loss: 0.8962221

Accuracy: 0.45
Loss: 0.6911929

Accuracy: 0.45
Loss: 0.79622155

Accuracy: 0.4
Loss: 0.8892376

Accuracy: 0.65
Loss: 0.7012502

Accuracy: 0.45
Loss: 0.83225596

Accuracy: 0.45
Loss: 0.8512502

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.55
Loss: 0.7625925

Accuracy: 0.55
Loss: 0.7632616

Accuracy: 0.5
Loss: 0.8132616

Accuracy: 0.55
Loss: 0.80125016

Accuracy: 0.45
Loss: 0.8632616

Accuracy: 0.4
Loss: 0.9322556

Accuracy: 0.3
Loss: 0.98225594

Accuracy: 0.5
Loss: 0.858233

Accuracy: 0.5
Loss: 0.83225596

Accuracy: 0.45
Loss: 0.8082328

Accuracy: 0.6
Loss: 0.70320433

Accuracy: 0.4
Loss: 0.7456735

Accuracy: 0.4
Loss: 0.7911929

Accuracy: 0.6
Loss: 0.7272457

Accuracy: 0.45
Loss: 0.72918147

Accuracy: 0.55
Loss: 0.7911929

Accuracy: 0.55
Loss: 0.7101872

Accuracy: 0.45
Loss: 0.73088443

Accuracy: 0.6
Loss: 0.72918147

Accuracy: 0.35
Loss: 0.72918147

Accuracy: 0.75
Loss: 0.6291815

Accuracy: 

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.7241529

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.2
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.4
Loss:

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.8
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.69

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.69314

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 

...

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.5
Loss: 

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.8
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.65
Loss:

Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accura

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.65
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.75
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.7
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.35
Loss: 0.6931472

Accuracy: 0.3
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.25
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.4
Loss: 0.6931472

Accuracy: 0.5
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.6
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.45
Loss: 0.6931472

Accuracy: 0.55
Loss: 0.693

In [7]:
x = tf.placeholder(tf.float32, [None, 1024]) 
y = tf.placeholder(tf.float32, [None, 2])
learning_rate = 1e-4
x_test = np.reshape(x_test, (-1, 1024))

def multilayer_perceptron(x, weights, biases): 
    # 隐藏层采用ReLU激活函数 
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1']) 
    layer_1 = tf.nn.relu(layer_1) 
    # 采用ReLU激活函数 
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']) 
    layer_2 = tf.nn.relu(layer_2) 
    
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3']) 
    layer_3 = tf.nn.relu(layer_3) 
    
    layer_4 = tf.add(tf.matmul(layer_3, weights['h4']), biases['b4']) 
    layer_4 = tf.nn.relu(layer_4) 
    # 输出层采用线性激活函数 
    out_layer = tf.matmul(layer_4, weights['out']) + biases['out'] 
    return out_layer 
    #返回输出层的值

    
weights = { 'h1': tf.Variable(tf.random_normal([1024, 512])), 
           'h2': tf.Variable(tf.random_normal([512, 256])), 
           'h3': tf.Variable(tf.random_normal([256, 128])),
           'h4': tf.Variable(tf.random_normal([128, 32])),
           'out': tf.Variable(tf.random_normal([32, 2])) } 

biases = { 'b1': tf.Variable(tf.random_normal([512])), 
          'b2': tf.Variable(tf.random_normal([256])), 
          'b3': tf.Variable(tf.random_normal([128])),
          'b4': tf.Variable(tf.random_normal([32])), 
          'out': tf.Variable(tf.random_normal([2])) } 

pred = multilayer_perceptron(x, weights, biases) 

logits = tf.nn.softmax(pred) 

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y)) 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)   



correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
#得出通过正确个数除以总数得出准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


    
# 训练集和测试集的随机分割

init = tf.global_variables_initializer()
with tf.Session() as sess: 
    sess.run(init) 
    #初始化参数 
    # Training cycle 
    for epoch in range(100000): 
        batch_index = random.randint(0, 1000) 
        #每一次从0-1000随机选出一个数，作为起始点 
        batch_data = x_train[batch_index:batch_index + 32] 
        batch_labels = y_train[batch_index:batch_index + 32] 
        #在上面所选的起始点开始，拿出32组数据进行一次训练 
        batch_data = np.reshape(batch_data, (-1, 1024))
        sess.run([optimizer, cost], feed_dict={x: batch_data, y: batch_labels})
        #进行训练 
        if epoch%500 == 0: 
            print(epoch)
            #accuracy = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
            print("Accuracy:", sess.run(accuracy, {x:x_test,y: y_test}))
            print("Loss:", sess.run(cost, {x:x_test,y: y_test}))
            
            #每隔300次 输出对应的次数以及测试集所测试的准确度 
    # Test model 
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1)) 
    # Calculate accuracy 
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    print("Total Accuracy:", accuracy.eval({x:x_test, y:y_test})) 
    #最后，当训练结束时，输出最终的准确率。

0
Accuracy: 0.54395604
Loss: 4628895.0
500
Accuracy: 0.52747256
Loss: 2543713.0
1000
Accuracy: 0.510989
Loss: 2189100.2
1500
Accuracy: 0.51648355
Loss: 2090709.5
2000
Accuracy: 0.510989
Loss: 2051600.5
2500
Accuracy: 0.521978
Loss: 2039032.5
3000
Accuracy: 0.52747256
Loss: 2014430.2
3500
Accuracy: 0.53296703
Loss: 2027084.5
4000
Accuracy: 0.53296703
Loss: 2028237.8
4500
Accuracy: 0.53296703
Loss: 2028237.8
5000
Accuracy: 0.53296703
Loss: 2028237.8
5500
Accuracy: 0.53846157
Loss: 2049763.8
6000
Accuracy: 0.53296703
Loss: 2092062.8
6500
Accuracy: 0.521978
Loss: 2019540.9
7000
Accuracy: 0.52747256
Loss: 2017040.4
7500
Accuracy: 0.52747256
Loss: 2017040.4
8000
Accuracy: 0.52747256
Loss: 2017040.4
8500
Accuracy: 0.52747256
Loss: 2017040.4
9000
Accuracy: 0.52747256
Loss: 2017040.4
9500
Accuracy: 0.52747256
Loss: 2017040.4
10000
Accuracy: 0.52747256
Loss: 2017040.4
10500
Accuracy: 0.52747256
Loss: 2017040.4
11000
Accuracy: 0.52747256
Loss: 2017040.4
11500
Accuracy: 0.52747256
Loss: 2017040.4


KeyboardInterrupt: 